## CrewAI核心组件 - 知识库

+ 字符串知识库
+ URL知识库

### 基于字符串知识库

In [ ]:
from crewai import Agent, Task, Crew, Process, LLM
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource
from dotenv import load_dotenv
import os

load_dotenv()

# 创建知识源
content = "用户名是John。他30岁，住在旧金山。"
# 装载知识
string_source = StringKnowledgeSource(
    content=content,
    metadata={"source": "用户信息"},
)

# 创建一个温度为0的LLM确保确定性输出
llm = LLM(
    model="gpt-4o-mini",
    temperature=0,
    api_key=os.environ.get("OPENAI_API_KEY"),
    base_url=os.environ.get("OPENAI_API_BASE"),
)

# 创建一个带有知识库的智能体
agent = Agent(
    role="用于用户",
    goal="你了解用户的一切。",
    backstory="你精通理解人们及其偏好。",
    llm=llm,
    verbose=True,
    allow_delegation=False,
)

task = Task(
    description="回答以下关于用户的问题：{question}",
    agent=agent,
    expected_output="对问题的回答",
)

crew = Crew(
    agents=[agent],
    tasks=[task],
    verbase=True,
    process=Process.sequential,
    knowledge_sources=[
        string_source
    ],  # 通过在这里添加源来启用知识。你也可以向源列表添加更多源。
)

result = crew.kickoff(inputs={"question": "John住在哪个城市，他多大年龄？"})

### 基于URL知识库

In [ ]:
! pip install docling

In [ ]:
from crewai import LLM, Agent, Crew, Process, Task
from crewai.knowledge.source.crew_docling_source import CrewDoclingSource

# 创建知识源
content_source = CrewDoclingSource(
    file_paths=[
        "https://lilianweng.github.io/posts/2024-11-28-reward-hacking",
        "https://lilianwengqithub.io/posts/2024-07-07-hallucination",
    ],
)
# 创建一个温度为0的LLM以确保确定性输出
llm = LLM(model="gpt-4o-mini", temperature=0)

# 创建一个带有知识库的智能体
agent = Agent(
    role="关于论文",
    goal="你了解论文的一切。",
    backstory="你精通理解论文及其内容。",
    llm=llm,
    verbose=True,
    allow_delegation=False,
)

task = Task(
    description="回答以下关于论文的问题:{question}",
    expected_output="对问题的回答。",
    agent=agent,
)
crew = Crew(
    agents=[agent],
    tasks=[task],
    verbose=True,
    process=Process.sequential,
    knowledge_sources=[
        content_source
    ],  # 通过在这里添加源来启用知识。你也可以向源列表添加更多源。
)

result = crew.kickoff(inputs={"question": "奖励黑客论文是关于什么的?请确保提供来源。"})